## **1. CHAT MODELS**

```
Khởi tạo 1 llm để chat 
```

In [3]:
from langchain_groq import ChatGroq
import os

llm = ChatGroq(model="llama3-70b-8192", 
                api_key=os.getenv("GROQ_API_KEY"),
                temperature=0.1)

response = llm.invoke("Thủ đô của nước Việt Nam là gì ? ")
print(response.content)

Thủ đô của nước Việt Nam là Hà Nội.


```Thêm instruction cho mô hình```

In [4]:
from langchain.schema.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Bạn là mô hình ngôn ngữ lớn ChatGPT"),
    HumanMessage(content="Bạn được phát triển bởi công ty nào ? Lưu ý: Chỉ được sử dụng tiếng Việt dể trả lời.")
]

response = llm.invoke(messages)
print(response.content)

Tôi được phát triển bởi công ty OpenAI, một công ty công nghệ trí tuệ nhân tạo có trụ sở tại San Francisco, California, Hoa Kỳ.


## **2. PROMPTS**
```
Prompt là 1 phần thiết yếu trong việc hướng dẫn llm để tạo ra kết quả phù hợp và mạch lạc.
```

In [5]:
"""Sử dụng PromptTemplate từ Langchain để tạo 1 prompt"""
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "Hãy nói cho về món {item} tại {country} . Lưu ý: chỉ được sử dụng tiếng Việt để trả lời."
)

response = llm.invoke(prompt.format(item = "Phở Bò", country="Việt Nam"))
print(response.content)

Phở bò, một món ăn quốc hồn quốc túy của Việt Nam! Phở bò là một món ăn truyền thống, được ưa chuộng và phổ biến rộng rãi trên toàn quốc.

Phở bò được làm từ các nguyên liệu chính là bánh phở (được làm từ bột gạo), nước dùng (được ninh từ xương bò và các gia vị), thịt bò (thịt bò chín hoặc bò tái), và các gia vị như hành, gừng, chanh, ớt... Tùy vào từng vùng miền, phở bò có thể được chế biến và phục vụ theo nhiều cách khác nhau.

Tại Hà Nội, phở bò được coi là một món ăn đặc sản, với nước dùng trong veo, ngọt và thơm, bánh phở mềm và dai, thịt bò chín mềm và ngọt. Phở bò Hà Nội thường được phục vụ với các gia vị như chanh, ớt, hành, gừng, và một ít rau thơm.

Tại Sài Gòn, phở bò được chế biến với nước dùng đậm đà hơn, bánh phở dai hơn, và thịt bò được xào với các gia vị như tỏi, ớt, và sa tế. Phở bò Sài Gòn thường được phục vụ với các gia vị như chanh, ớt, hành, và một ít rau thơm.

Phở bò không chỉ là một món ăn ngon, mà còn là một phần của văn hóa và truyền thống Việt Nam. Phở bò đượ

In [7]:
from langchain.prompts import ChatPromptTemplate


"""Định nghĩa 1 prompt với nhiều vai trò khác nhau trong prompt sử dụng ChatPromptTemplate từ langchain."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Bạn là 1 trợ lý ảo hữu ích. Tên của bạn là {name}"),
        ("human", "Xin chào, bạn có khỏe không ?"),
        ("ai", "Tôi khỏe, cảm ơn bạn !"),
        ("human", "{user_input}")
    ]
)

formated_messages = prompt.format(name="Llama", user_input="Đất nước Việt Nam có bao nhiêu dân tộc ? Lưu ý: bạn chỉ được trả lời bằng tiếng việt.")
response = llm.invoke(formated_messages)
print(response.content)

Xin chào! Ở Việt Nam có 54 dân tộc anh em.


## **3. OUTPUT PARSERS**

```Trình phân tích cú pháp đầu ra đóng một vai trò quan trọng trong Langchain, cho phép người dùng cấu trúc các phản hồi do mô hình ngôn ngữ tạo ra.```

#### **3.1 PydanticOutputParser**

In [22]:
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator


# Init the language model
llm = ChatGroq(model="llama3-70b-8192", 
                api_key=os.getenv("GROQ_API_KEY"),
                temperature=0.1)

"""Định nghĩa cấu trúc dữ liệu mong muốn bằng Pydantic"""
class Joke(BaseModel):
    setup: str = Field()
    punchline: str = Field()

    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Câu hỏi không đúng định dạng")
        return field
    
"""Setup PydanticOuputParser"""
parser = PydanticOutputParser(pydantic_object=Joke)

"""Tạo 1 prompt"""
prompt = PromptTemplate(
    input_variables=['query'],
    template="Trả lời câu hỏi của người dùng.\n{format_instructions}\n{query}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

"""địng nghĩa 1 prompt"""
query = "Hãy hỏi 1 câu về Việt Nam. Lưu ý: Bạn chỉ được sử dụng tiếng việt"

"""combine prompt vs llm"""
chain = prompt | llm
output = chain.invoke({"query": query})

"""Phân tích cú pháp (parser) sử dụng parser"""
parsed_query = parser.invoke(output)
print("question: ", parsed_query.setup)
print("answer: ", parsed_query.punchline)




question:  Thành phố lớn nhất của Việt Nam là gì?
answer:  Hà Nội


#### **3.2 SimpleJsonOutputParser**

In [31]:
from langchain.output_parsers.json import SimpleJsonOutputParser

"""tạo 1 JSON prompt"""
json_prompt = PromptTemplate.from_template(
    "Trả về 1 JSON gồm chỉ gồm các khóa 'ngày sinh' và 'nơi sinh' để trả lời câu hỏi: {question}"
)

"""Khởi tạo 1 JSON parser"""
json_parser = SimpleJsonOutputParser()

"""Combine các thành phần lại với nhau"""
json_chain = json_prompt | llm | json_parser


response = list(json_chain.stream({"question": "Ngày tháng năm sinh và nơi sinh của Chủ tịch Hồ Chí Minh ? Lưu ý: Bạn chỉ được sử dụng tiếng Việt"}))

print(response[-1])
                

{'ngày sinh': '19/05/1890', 'nơi sinh': 'Hồ Chí Minh sinh tại làng Hoàng Trù, huyện Nam Đàn, tỉnh Nghệ An'}


#### **3.3 CommaSeparatedListOutputParser**

In [52]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from groq import Groq


"""Khởi tạo parser"""
output_parser = CommaSeparatedListOutputParser()

"""Tạo 1 hướng dẫn cho model"""
format_instruction = output_parser.get_format_instructions()

"""Tạo 1 prompt"""
prompt = PromptTemplate(
    input_variables=["subject"],
    template="1 danh sách gồm 5 {subject}\{format_instruction}. Lưu ý: Bạn chỉ được sử dụng tiếng việt dể  trả lời, chỉ đưa 2 5 subject, không cần gì thêm.",
    partial_variables={"format_instruction": format_instruction}
)

"""Tạo query"""
query = "5 đội bóng ở giải đấu ngoại hạng Anh"

"""Invoke"""
response = llm.invoke(prompt.format(subject=query))
"""Parser output using output_parser"""
output_parsed = output_parser.parse(response.content)

print(output_parsed)

['Manchester City', 'Liverpool', 'Chelsea', 'Arsenal', 'Tottenham']


#### **3.4 DatetimeOutputParser**

In [65]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import DatetimeOutputParser
from langchain.chains import LLMChain


"""Init the DatetimeOutputParser"""
output_parser = DatetimeOutputParser()

"""Tạo 1 template instruction"""

template = """
Câu trả lời cho câu hỏi của người dùng:
{question}
{format_instruction}
Lưu ý: Bạn chỉ được sử dụng tiếng việt dể  trả lời.
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template,
    partial_variables={"format_instruction": output_parser.get_format_instructions()},
)

"""Combine các thành phần lại với nhau"""
chain = prompt | llm

"""Định nghĩa 1 query"""
query = "Ngày xảy ra thế chiến thứ 2 là khi nào ?"

response = llm.invoke(prompt.format(question=query))

"""Parse output using datetime parser"""
parsed_output = output_parser.parse(response.content)

print(parsed_output)

1939-09-01 04:45:00
